In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns 

%load_ext autoreload
%matplotlib inline
%autoreload 2

In [ ]:
import warnings 
warnings.filterwarnings("default")

# Set plotting aesthetics

In [ ]:
mpl.rcParams.keys();

In [ ]:
mpl.rcParams.update({'font.size': 15})
mpl.rcParams.update({"axes.grid" : True, "grid.linestyle": '--', 
                     "grid.alpha": 0.8, "grid.color": "black"})
mpl.rcParams.update({"lines.linewidth" : 3})
mpl.style.use('seaborn-colorblind')

In [ ]:
def errorbar(M, x, label, ax, marker, color, ls='-'):
    n_runs = M.shape[1]
    M_means = np.mean(M, axis=1)
    M_std_normalized = np.std(M, axis=1)/np.sqrt(n_runs)
    M_upper = M_std_normalized
    M_lower = M_std_normalized
    if(M.shape[0] > 15):
        plot_every = 3
    else: 
        plot_every = 1
        
    ax.errorbar(x[::plot_every], M_means[::plot_every], np.vstack((M_lower[::plot_every], M_upper[::plot_every])), label=label, marker=marker, elinewidth=2, ms=8, ls=ls, color=color)
    if(M.shape[0] > 20):
        ax.set_xticks([5000, 15000, 25000])

In [ ]:
def plot(results, ax, ece=True):
    run = 0
    N_calib_points = results[run][12]
    n_runs = results.shape[0]
    ECE_1 = np.zeros((N_calib_points.size, n_runs))
    ECE_2 = np.zeros((N_calib_points.size, n_runs))
    ECE_3 = np.zeros((N_calib_points.size, n_runs))
    ECE_4 = np.zeros((N_calib_points.size, n_runs))
    ECE_5 = np.zeros((N_calib_points.size, n_runs))
    ECE_6 = np.zeros((N_calib_points.size, n_runs))
    
    SHP_1 = np.zeros((N_calib_points.size, n_runs))
    SHP_2 = np.zeros((N_calib_points.size, n_runs))
    SHP_3 = np.zeros((N_calib_points.size, n_runs))
    SHP_4 = np.zeros((N_calib_points.size, n_runs))
    SHP_5 = np.zeros((N_calib_points.size, n_runs))
    SHP_6 = np.zeros((N_calib_points.size, n_runs))

    for run in range(n_runs):
        ECE_1[:,run] = results[run][0]
        ECE_2[:,run] = results[run][1]
        ECE_3[:,run] = results[run][2]
        ECE_4[:,run] = results[run][3]
        ECE_5[:,run] = results[run][4]
        ECE_6[:,run] = results[run][5]

        SHP_1[:,run] = results[run][6]
        SHP_2[:,run] = results[run][7]
        SHP_3[:,run] = results[run][8]
        SHP_4[:,run] = results[run][9]
        SHP_5[:,run] = results[run][10]
        SHP_6[:,run] = results[run][11]

    colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']
    if(ece):
        errorbar(ECE_3, N_calib_points, "Windowed histogram binning (WHB)", ax, marker='o', ls='dotted', color=colors[0])
        errorbar(ECE_6, N_calib_points, "WHB + tracking (TWHB)", ax, marker='o', ls='solid', color=colors[0])
        errorbar(ECE_1, N_calib_points, "Online Platt scaling (OPS)", ax, marker='>', ls='dotted', color=colors[3])
        errorbar(ECE_4, N_calib_points, "OPS + tracking (TOPS)", ax, marker='>', ls='solid', color=colors[3])
        errorbar(ECE_2, N_calib_points, "Online beta scaling (OBS)", ax, marker='<', ls='dotted', color=colors[4])
        errorbar(ECE_5, N_calib_points, "OBS + tracking (TOBS)", ax, marker='<', ls='solid', color=colors[4])
        ax.set_ylabel(r'$\mathrm{\mathbb{E}}[$Calibration-error (CE) at $t]$')
        
    else: 
        errorbar(SHP_3, N_calib_points, "Windowed histogram binning (WHB)", ax, marker='o', ls='dotted', color=colors[0])
        errorbar(SHP_6, N_calib_points, "WHB + tracking (TWHB)", ax, marker='o', ls='solid', color=colors[0])
        errorbar(SHP_1, N_calib_points, "Online Platt scaling (OPS)", ax, marker='>', ls='dotted', color=colors[3])
        errorbar(SHP_4, N_calib_points, "OPS + tracking (TOPS)", ax, marker='>', ls='solid', color=colors[3])
        errorbar(SHP_2, N_calib_points, "Online beta scaling (OBS)", ax, marker='<', ls='dotted', color=colors[4])
        errorbar(SHP_5, N_calib_points, "OBS + tracking (TOBS)", ax, marker='<', ls='solid', color=colors[4])
        ax.set_ylabel(r'$\mathrm{\mathbb{E}}[$Sharpness (SHP) at $t]$')

    ax.legend(loc='upper center', bbox_to_anchor=(-0.1, 1.35),
                     ncol=3, fancybox=True, shadow=True)
    ax.set_xlabel(r'Time t $\rightarrow$')

In [ ]:
results = np.load("results_hb/real_data/bank_0.npy")
fig, ax = plt.subplots(figsize=(5,4))
plot(results, ax)

In [ ]:
dataset_name = {'bank': "Bank marketing response",
               'credit': "Credit default", 
               'churn': "Customer churn modeling", 
               'fetal': "Fetal Health"}

def create_final_plots(dataset, design, ece=True):
    result_file = "results_hb/real_data/{}_{}.npy".format(dataset, design)
    results = np.load(result_file)
    fig, ax = plt.subplots(figsize=(5,4))
    plot(results, ax, ece)
    if(ece):
        fig_file = "results_hb/real_data_figures/{}_{}.pdf".format(dataset, design)
    else:
        fig_file = "results_hb/real_data_figures/{}_{}_sharpness.pdf".format(dataset, design)
    plt.title(dataset_name[dataset])
    plt.savefig(fig_file, bbox_inches="tight")

In [ ]:
for dataset in dataset_name.keys():
    create_final_plots(dataset, 1)